## Bring in dependencies

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

In [19]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings

In [28]:
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec


## Create basic tools

In [4]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [29]:
finance_tools = YahooFinanceToolSpec().to_tool_list()

In [30]:
finance_tools.extend([multiply_tool, add_tool])

## Initialize the LLM

In [20]:
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
# from llama_index.llms.ollama import Ollama
# llm = Ollama(model="deepseek-r1:7b", request_timeout=360.0)
# llm = Ollama(model="qwen2.5:3b", request_timeout=360.0)



## Load and index documents

In [21]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [22]:
response = query_engine.query(
    "What was the total amount of the 2023 Canadian federal budget?"
)
print(response)

The total amount of the 2023 Canadian federal budget was projected at $496.9 billion in expenditures.


## Add a query engine tool

In [23]:
from llama_index.core.tools import QueryEngineTool

In [24]:
budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget.",
)

## Initialize the agent

In [31]:
# agent = ReActAgent.from_tools(
#     [multiply_tool, add_tool, budget_tool], verbose=True
# )

agent = ReActAgent.from_tools(finance_tools, verbose=True)

response = agent.chat("What is the current price of NVDA?")

print(response)

> Running step e2d0a3de-8ec0-47c4-be8c-a68348676551. Step input: What is the current price of NVDA?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: stock_basic_info
Action Input: {'ticker': 'NVDA'}
Observation: Info: 
{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPU

In [32]:
finance_tools

## Ask a question

In [26]:
response = agent.chat(
    "What is the total amount of the 2023 Canadian federal budget multiplied by 3? Go step by step, using a tool to do any math."
)

print(response)

> Running step 8fe1d546-989d-4000-8536-129c7591ac56. Step input: What is the total amount of the 2023 Canadian federal budget multiplied by 3? Go step by step, using a tool to do any math.
Thought: The current language of the user is: English. I need to use a tool to find the total amount of the 2023 Canadian federal budget first before performing the multiplication.
Action: canadian_budget_2023
Action Input: {'input': 'What is the total amount of the 2023 Canadian federal budget?'}
Observation: The total amount of the 2023 Canadian federal budget is projected to be $496.9 billion in expenditures.
> Running step c71de002-4425-48ec-a448-dcdcff788ebd. Step input: None
Thought: I have the total amount of the 2023 Canadian federal budget, which is $496.9 billion. Now I will multiply this amount by 3.
Action: multiply
Action Input: {'a': 496.9, 'b': 3}
Observation: 1490.6999999999998
> Running step c50a03c6-96f5-4daa-b871-6a46e7646eee. Step input: None
Thought: I can answer without using an

## Memory

In [27]:
response = agent.chat(
    "How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?"
)

print(response)

response = agent.chat(
    "How much was allocated to a implement a means-tested dental care program in the 2023 Canadian federal budget?"
)

print(response)

response = agent.chat(
    "How much was the total of those two allocations added together? Use a tool to answer any questions."
)

print(response)

> Running step d2f5a173-3316-4512-a628-9cbf737ab9a7. Step input: How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: canadian_budget_2023
Action Input: {'input': 'How much was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?'}
Observation: The specific amount allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget is not mentioned in the provided information.
> Running step e9b25e99-5137-48df-a6c6-eb08117bd741. Step input: None
Thought: I cannot answer the question with the provided tools.
Answer: The specific amount allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget is not available in the provided information.
The specific amount alloc